## ToDo
1. Search implementation to interpret using OpenAI: Output keywords
    a. Tokenizer
2. Scraper to create dataset of AI from website along with keywords
3. 

## Job LinkedIn
1. Target people "Open for Work"
2. LinkedIn API - People Search 

## People + AI


In [ ]:

Save New Duplicate & Edit Just Text Twitter
To use the NLTK library to tokenize a text and check it with a Firebase database, you'll need to follow these steps:

Install the NLTK library: NLTK can be installed using pip, the Python package installer. Open your terminal or command prompt and run the following command:

Copy code
pip install nltk
Import the necessary modules: In your Python script, import the required modules for NLTK and Firebase. Here's an example:

python
Copy code
import nltk
from nltk.tokenize import word_tokenize
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
Initialize the Firebase app and connect to your database: Before interacting with your Firebase database, you need to initialize the Firebase app and establish a connection. Here's an example of how to initialize the app using a service account credential:

python
Copy code
# Replace 'path/to/serviceAccountKey.json' with the path to your service account key file
cred = credentials.Certificate('path/to/serviceAccountKey.json')
firebase_admin.initialize_app(cred)

# Connect to the Firestore database
db = firestore.client()
Tokenize the input text: Use the word_tokenize() function from NLTK to tokenize the input text from the search bar. Here's an example:

python
Copy code
input_text = input("Enter the text: ")  # Get the input text from the user

tokens = word_tokenize(input_text)
Query the Firebase database: Use the tokenized text to query your Firebase database and check if it exists. You can use the Firestore API to perform the query. Here's an example:

python
Copy code
# Replace 'collection_name' with the name of your collection in Firebase
collection_ref = db.collection('collection_name')

# Query the database for the tokens
query = collection_ref.where('token', 'in', tokens).limit(1).get()

# Check if the query returned any results
if len(query) > 0:
    print("Text exists in the database!")
else:
    print("Text does not exist in the database.")
Remember to replace 'path/to/serviceAccountKey.json' with the actual path to your Firebase service account key file and 'collection_name' with the name of your collection in Firebase.

Make sure you have the necessary permissions and authentication set up in your Firebase project to access the database.

# LinkedIn Api implementation

In [ ]:
import requests

# Set your access token
access_token = 'YOUR_ACCESS_TOKEN'

# Set the search query parameters
query = 'React developer'  # Your search query
num_results = 10  # Number of results to retrieve
start = 0  # Start index of the search results

# Set the API endpoint URL
api_url = f'https://api.linkedin.com/v2/search/people?q={query}&count={num_results}&start={start}'

# Set the request headers
headers = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json',
}

# Send the GET request to the API endpoint
response = requests.get(api_url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response data
    data = response.json()
    print(data)
else:
    # Handle error response
    print(f'Request failed with status code: {response.status_code}')
    print(response.text)

# Scraper

In [41]:
import re
from collections import OrderedDict
import json
import textstat
import wikipedia
import urllib.parse
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse
from bs4.element import NavigableString

def calculate_reading_time(text):
    # Use the textstat library to calculate the reading time
    return textstat.reading_time(text)

def find_image_url(tag):
    if tag.name == 'img':
        return tag['src']
    else:
        for child in tag.children:
            if child.name:
                img_url = find_image_url(child)
                if img_url:
                    return img_url
                
def check_and_modify_url(url, img_url):
    if img_url != None:
        parsed_url = urllib.parse.urlparse(url)
        parsed_img_url = urllib.parse.urlparse(img_url)
        
        # Handle missing scheme and netloc
        if not parsed_img_url.scheme and not parsed_img_url.netloc:
            img_url = f"{parsed_url.scheme}://{parsed_url.netloc}{img_url}"
            
        # Handle missing scheme
        elif not parsed_img_url.scheme:
            img_url = f"{parsed_url.scheme}:{img_url}"
            
        # Handle missing netloc
        elif not parsed_img_url.netloc:
            img_url = parsed_img_url._replace(netloc=parsed_url.netloc).geturl()
            
        return img_url
    
def format_string(input_string):
    # Replace elements of the form "{\displaystyle <element>}"
    # with the corresponding element surrounded by "$"s
    output_string = re.sub(r'(\S*)\{\\displaystyle (\S* )\}', r'\\\(\\\2\\\)', input_string)
    output_string = re.sub(r'(\S )\{\\displaystyle (\S*)\}', r'$\1$', output_string)
    output_string = re.sub(r'(\S*)\{\\displaystyle (\S*)\}', r'$\1$', output_string)
    
    return output_string


def extract_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    if 'www.' in domain:
        domain = domain.replace('www.', '')
    return domain
    

DELIMITER = '\n'

def extract_source(url):
	agent = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
	source=requests.get(url, headers=agent).text
	return source

def extract_data(source, unwanted, url):
    soup = BeautifulSoup(source, 'html.parser')

    headings = soup.find_all(re.compile("^h[1-6]$"))

    headings_dict = OrderedDict()
    hierarchy = [headings_dict]
    current_dict = headings_dict

    unwanted = [x.lower() for x in unwanted]

    for heading in headings:
        level = int(heading.name[1])

        while level < len(hierarchy):
            hierarchy.pop()
            current_dict = hierarchy[-1]

        heading_text = heading.text.strip().replace('[edit]', '').replace('\u2022', '')

        if heading_text.lower() in unwanted:
            continue

        current_dict[heading_text] = OrderedDict()
        last_heading = heading_text
        current_dict = current_dict[heading_text]
        hierarchy.append(current_dict)

        paragraphs = []

        current_element = heading

        while True:
            next_element = current_element.next_sibling

            if next_element and (next_element.name == 'p'):
                paragraph_text = ""
                heading_found = False

                for content in next_element.contents:
                    if content.name == 'a':
                        heading_text = content.text.strip()
                        heading_found = True
                        current_dict[heading_text] = OrderedDict()
                        current_dict[heading_text]["url"] = content["href"]

                        # Extract the definition after the hyphen "-"
                        definition = next_element.text.strip().split("-", 1)[-1].strip()
                        current_dict[heading_text]["definition"] = definition
                        current_dict[heading_text]["features"] = ""
                        break

            elif next_element and next_element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'a']:
                break

            elif next_element is None:
                break

            current_element = next_element

    headings_dict = json.dumps(headings_dict, indent=4)
    return headings_dict






dictionaries = []

# data_read = ref.get()
unwanted = []
print(unwanted)

[]


In [42]:
url = "https://fentolia.github.io/AI-Infinity/#spreadsheets"
d = extract_data(extract_source(url), unwanted, url)
print(d)

{
    "AI-Infinity": {
        "AI-Infinity - A set of cutting-edge AI tools for experimenting with new ideas\ud83d\udca1technologies\ud83c\udf10, and algorithms\ud83d\udc68\u200d\ud83d\udcbb.": {
            "": {
                "url": "https://twitter.com/intent/tweet?url=https%3A%2F%2Fgithub.com%2Fmeetpateltech%2FAI-Infinity&text=If%20you%27re%20looking%20for%20the%20latest%20and%20greatest%20AI%20tools%2C%20you%20won%27t%20want%20to%20miss%20the%20AI%20Infinity%20GitHub%20repository.%20Check%20it%20out%20now%21",
                "definition": ""
            }
        }
    },
    "\u26a0\ufe0fUpdate:": {
        "Categories :": {
            "AI Detection \ud83d\udd75\ufe0f": {
                "url": "#ai-detection",
                "definition": "AI Detection \ud83d\udd75\ufe0f | Audio \ud83d\udce3 | Avatars \ud83d\udc64 | Chat \ud83d\udcac | Coding \ud83d\udda5\ufe0f | Copywriting \u2328\ufe0f | Customer Support \ud83d\udc81 | Design Assistant \ud83c\udfa8 | Developer \ud83d\udc